# Calculos y Resultados

## Validación de modelos en *ANSYS*

### *Caída hidráulica*

Para este caso los resultados de la modelación en **ANSYS** son consistente con los de las ecuaciones teóricas, en los dos casos se presentan bajas velocidades en la parte aguas arriba de la caída, con *números de Froude* menores a 1, y altas velocidades en la parte aguas abajo de la caída, con *números de Froude* superiores a 1. La siguiente figura ilustra la solución en **ANSYS**, muestra la caída hidráulica y los vectores de velocidad en la fase agua, también una escala de colores para la magnitud de velocidad.

<img src="figura6.jpg" width="600">


****Fig 6. Esquema de caída hidráulica modelado en *ANSYS*.****


Se comparó los resultados de velocidad en la caída hidráulica, la calculada con las ecuaciones teóricas y la arrojada por el modelo aproximadamente en el punto de inflexión en la curva inversa, también se comparó la profundidad de flujo.

La velocidad calculada por las ecuaciones teóricas es de 3.18 m/s, y la modelada en **ANSYS** es aproximadamente 3.10  m/s, corresponde a la velocidad crítica en el punto de inflexión de la curva inversa.

A continuación, los calculos de la profundidad de flujo con las ecuaciones teóricas y la modelada en **ANSYS**. Se presentan errores que oscilan entre el 6% y el 12%, tomando como real la profundidad calculada con las ecuaciones teóricas.


In [1]:
import numpy as np
from sympy import *
import pandas as pd
import plotly.graph_objects as go

x = Symbol ('x')
y = Symbol ('y')

# datos
a0 = 0.866 # ancho del canal en la contracción en metros 'sección critica'
a1 = 3.048 # ancho del canal aguas arriba en metros
a2 = 2.438 # ancho del canal aguas abajo en metros
l = 15.22 # longitud del canal en metros
y1 = 1.52 # profundidad del flujo aguas arriba en metros
q = 2.83 # caudal en m3/s
g = 9.81 # gravedad en m2/s

# solución
E = y1 + q**2/(2*g*(a1*y1)**2) # Energia total, se mantiene constante a lo largo del canal, pérdidas de energía insignificante. 
yc = E/1.5 # Profundidad crítica
vc = (yc*g)**(0.5) # Velocidad crítica
ac = q/(vc*yc) # Ancho de la sección crítica

print ("Energía(m):", round(E, 2) , " yc(m):", round(yc, 2), " Vc(m/s):", round(vc, 2), " ancho de la sección critica(m):", round(ac, 2))

# Cálculo del perfil
a = []
a = np.empty(26, dtype=object)
for i in range(0, 13):
     a[i] = 3.05 - 2*((i)*0.762*1.09/9.144)
     a[i+13] = 0.8699999999999997 + 2*((i+1)*0.762*0.785/9.144)

x0 = []
x0 = np.empty(25, dtype=object)
for i in range(0, 25):
     x0[i] = (i)*0.762

y2 = []
y2 = np.empty(26, dtype=object)

for i in range(0, 13):
     resultado = solve([y**3-E*y**2+q**2/(2*g*(a[i])**2)], [a[i], y])
     y3 = resultado[2]
     y2[i] = float(round(y3[1],3))
     resultado2 = solve([y**3-E*y**2+q**2/(2*g*(a[i+13])**2)], [a[i+13], y])
     y4 = resultado2[1]
     y2[i+13] = float(round(y4[1],3))

# Resultados ANSYS
y5 = [1.5792, 1.56, 1.5826, 1.5819, 1.5723, 1.5674, 1.5392, 1.4824, 1.4506, 1.4397, 1.0310, 0.6740, 0.5554, 0.4676, 0.399, 0.3398, 0.3058, 0.1984, 0.2356, 0.1301, 0.2079]

caida_hid = {'ancho_canal': a[2:23], 'longitud': x0[2:23], 'yteorica': y2[2:23], 'yansys': y5}
caida_hid = pd.DataFrame(data=caida_hid)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=caida_hid['longitud'], y=caida_hid['yteorica'],
                    mode='lines',
                    name='Teórica',
                    line=dict(color='royalblue', width=4, dash='dot')
                    ))
fig.add_trace(go.Scatter(x=caida_hid['longitud'], y=caida_hid['yansys'],
                    mode='lines+markers',
                    name='ANSYS',
                    line=dict(color='firebrick', width=4, dash='dot')
                    ))
# Edit the layout
fig.update_layout(title='Comparación profundidad de flujo ANSYS vs Teórico',
                   xaxis_title='longitud canal (m)',
                   yaxis_title='Profundidad del flujo (m)')

fig.show()


Energía(m): 1.54  yc(m): 1.03  Vc(m/s): 3.17  ancho de la sección critica(m): 0.87
